In [ ]:
import pandas as pd

# df_train = pd.read_csv("vuln_data_train.csv")
# df_train

df_train = pd.read_csv("vuln_data_val.csv")
df_train

,file_name,vuln_title,vuln_explanation,severity,vuln_recommendation,vuln_code
0,123.md,`AuraBalRewardPool` charges a penalty to all u...,"When claiming their rewards, users are charged...",medium,Don't charge the penalty if the locker has bee...,"function _lock(address _account, uint256 _amou..."
1,106.md,Might not get desired min loan amount if `_ori...,Admins can update the origination fee by calli...,medium,"Reconsider how the min loan amount works. Imo,...",function createLoan(\n uint256 collater...
2,41.md,Basket becomes unusable if everybody burns the...,"While handling the fees, the contract calculat...",medium,Add a check to `handleFees: if totalSupply= 0`...,function handleFees() private {\n if (l...
3,5.md,Out-of-bound index access in function `getAnch...,Out-of-bound index access is possible in the f...,low,Consider using `arrayPrices.length/2` as the i...,function getAnchorPrice() public view returns ...
4,70.md,Oracle returns an improperly scaled USDV/VADER...,The LBT oracle does not properly scale values ...,high,Go over oracle calculation again to ensure tha...,function _calculateUSDVPrice(\n uint256...
...,...,...,...,...,...,...
137,114.md,Owner or Managers can rug Aave rewards,A malicious owner or manager can steal all Aav...,medium,Use a poolAddressesProviderRegistry-like contr...,function claimRewards(address _to) external on...
138,3.md,Different solidity version in UniswapStyleLib.sol,The solidity version in UniswapStyleLib.sol (>...,low,It is cleaner to have the same solidity versio...,function getAmountOut(\n uint256 amount...
139,52.md,Incorrect Accrual Of `sumNative` and `sumUSD` ...,The `TwapOracle.consult()` function iterates o...,high,To calculate the correct consultation of a giv...,function consult(address token) public view re...
140,35.md,Wrong usage of `positionId` in `ConcentratedLi...,In the `subscribe` function of `ConcentratedLi...,high,Change `positionId` to `incentiveId` in the re...,"function subscribe(uint256 positionId, uint256..."


In [ ]:
import re
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from vllm import SamplingParams

def extract_answer(text: str) -> str | None:
    answer = text.split("</think>\n\n")[-1]
    if "<think>" in answer:
        return None
    return answer.strip()

# Sampling parameters
sampling_params = SamplingParams(
    temperature=0.7,
    top_k=20,
    top_p=0.8,
    min_p=0,
    max_tokens=8192,
)

THINKER_SYSTEM_PROMPT = [
    """You are an LLM Chain-of-Thought Generator. That answer straight the Chain-of-Thought without any commentary or additional information.""",
    """You are a code analyst specialized in security. Your job is to provide a detailed Chain-of-Thought analysis without adding explanations or commentary outside the logical reasoning steps.""",
    """You are a reviewer performing a vulnerability walk-through. Just present your thought process step-by-step without external opinions or extra commentary.""",
    """You are a vulnerability detective. Present only your internal reasoning steps that logically lead from the code to the known issue. No narration or conclusions outside of your investigation.""",
    """You're a mentor walking a junior auditor through a known vulnerability discovery. Only share your logical process without any fluff, summaries, or external commentary."""
]

THINKER_USER_PROMPT = [
    """Given the vulnerable solidity code and the ground truth vulnerability in the context.
Your task is to give Chain-of-Thought how we can caught the vulnerability just from the code.
Your Chain-of-Thought should has: the understanding the use case of the code, analyzing the vulnerabilities and give the final vulnerability answer that is the ground truth vulnerability.
Give your answer as a human thinking, so start with something like: "Okay so we have ...", "Okay lets see...", etc.

**Vulnerable Code:**
```solidity
{code}```

**Ground Truth Vulnerability:**  
{ground_truth}
""",
    
    """You are given a vulnerable Solidity smart contract and the known ground truth vulnerability.  
Your task is to think through how a security engineer would identify that specific vulnerability just by looking at the code.  

Structure your reasoning like natural human thinking, with transitions such as:  
"Alright, the first thing I notice...", "If I follow the logic here...", etc. 
And start with something like:
"Okay so we have ...", "Okay lets see...", etc. 

**Code:**  
```solidity
{code}```

**Ground Truth Vulnerability:**  
{ground_truth}
""",
    
    """You're reviewing this smart contract with the aim of discovering a specific known vulnerability.

Explain how you'd find the vulnerability by thinking through the code.  
Keep the tone human and reflective, as if explaining to a colleague. Start with lines like:  
"Okay, I'm looking at this function...", "Okay lets check the function....At first glance, this looks fine, but...", etc. 

**Smart Contract Code:**  
```solidity
{code}```

**Target Vulnerability (Ground Truth):**  
{ground_truth}
""",
    
    """Imagine you're investigating this Solidity contract like a detective solving a case.

Work your way toward the vulnerability step-by-step. Let your Chain-of-Thought feel like natural human deduction. Use transitions like:  
"Hmm, this function seems suspicious...", "Wait a second, if this variable isn’t protected...", etc.  
And start with something like "Okay lets check the functionality...", "Okay lets take a look the function", etc. 

**Vulnerable Code:**  
```solidity
{code}```

**Known Flaw (Ground Truth):**  
{ground_truth}
""",
    
    """You are mentoring someone in finding vulnerabilities. You already know what the issue is, but you're walking them through how you'd discover it naturally just from reading the Solidity code.

Make your Chain-of-Thought sound human and intuitive
"Okay let’s take a look at how this function behaves...", "Okay lets notice how this variable is exposed...", etc. 

**Review Code:**  
```solidity
{code}```

**Known Vulnerability (Ground Truth):**  
{ground_truth}
"""
]

thinker_model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
thinker_max_seq_length = 8192  
thinker_load_in_4bit = True 

thinker_model, thinker_tokenizer = FastLanguageModel.from_pretrained(
    model_name=thinker_model_name,
    load_in_4bit = thinker_load_in_4bit,
    fast_inference=True,
    max_seq_length=thinker_max_seq_length,
)
FastLanguageModel.for_inference(thinker_model) # Enable native 2x faster inference


def llm_thinker(idx, code, ground_truth):
    messages = [
        {"role": "system", "content": THINKER_SYSTEM_PROMPT[idx]},
        {"role": "system", "content": THINKER_USER_PROMPT[idx].format(code=code, ground_truth=ground_truth)},
    ]
    
    # Apply chat template
    text = thinker_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
        enable_thinking=False,
    )
    
    try:
        # Generate output
        output = thinker_model.fast_generate(
            [text],
            sampling_params=sampling_params,
        )[0].outputs[0].text
    except Exception as e:
        print(f"Error generating {idx}: {e}")
        return None
    
    return extract_answer(output)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...

INFO 05-30 04:18:04 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 05-30 04:18:04 [model_runner.py:1140] Model loading took 10.8959 GiB and 7.805400 seconds
INFO 05-30 04:18:11 [worker.py:287] Memory profiling takes 6.29 seconds
INFO 05-30 04:18:11 [worker.py:287] the current vLLM instance can use total_gpu_memory (31.75GiB) x gpu_memory_utilization (0.49) = 15.70GiB
INFO 05-30 04:18:11 [worker.py:287] model weights take 10.90GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 1.36GiB; the rest of the memory reserved for KV Cache is 3.38GiB.
INFO 05-30 04:18:11 [executor_base.py:112] # cuda blocks: 1382, # CPU blocks: 2457
INFO 05-30 04:18:11 [executor_base.py:117] Maximum concurrency for 8192 tokens per request: 2.70x
INFO 05-30 04:18:16 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decre

Capturing CUDA graph shapes: 100%|█████████████████████████████████████████████████████████| 27/27 [00:36<00:00,  1.34s/it]

INFO 05-30 04:18:52 [model_runner.py:1592] Graph capturing finished in 36 secs, took 0.94 GiB
INFO 05-30 04:18:52 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 48.00 seconds


Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


In [ ]:
import os
import pandas as pd
import random
from tqdm import tqdm

# output_path = 'vuln_data_train_with_CoT.csv'
output_path = 'vuln_data_val_with_CoT.csv'

# Load existing processed rows if the file already exists
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    processed_indices = set(df_existing.index)
else:
    df_existing = pd.DataFrame()
    processed_indices = set()

# Prepare 5 columns in the original df if not already present
for i in range(5):
    if f'CoT_{i}' not in df_train.columns:
        df_train[f'CoT_{i}'] = None

# Process and append to CSV one by one
for idx in tqdm(range(len(df_train))):
    if idx in processed_indices:
        print(f"Skipped Ids: {idx}")
        continue  # Skip already processed rows

    answers = []
    for i in range(5):
        answer = llm_thinker(i, df_train.loc[idx, 'vuln_code'], df_train.loc[idx, 'vuln_explanation']) 
        answers.append(answer)

    random.shuffle(answers)

    for i, ans in enumerate(answers):
        df_train.at[idx, f'CoT_{i}'] = ans

    # Extract just the current row and append to file
    row_df = df_train.loc[[idx]]  # Double brackets to keep it as a DataFrame
    row_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=True)

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.66s/it, est. speed input: 86.83 toks/s, output: 29.19 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:24<00:00, 24.03s/it, est. speed input: 38.67 toks/s, output: 30.22 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:32<00:00, 32.10s/it, est. speed input: 28.45 toks/s, output: 30.41 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.52s/it, est. speed input: 80.55 toks/s, output: 29.51 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:09<00:00,  9.97s/it, est. speed input: 169.13 toks/s, output: 27.89 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:14<00:00, 14.11s/it, est. speed input: 119.80 toks/s, output: 28.78 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.79s/it, est. speed input: 84.59 toks/s, output: 29.36 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:24<00:00, 24.32s/it, est. speed input: 69.45 toks/s, output: 29.61 toks/s]



Processed prompts: 100%|██████████████| 1/1 [00:14<00:00, 14.27s/it, est. speed input: 203.49 toks/s, output: 27.06 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:19<00:00, 19.06s/it, est. speed input: 152.54 toks/s, output: 27.97 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:27<00:00, 27.44s/it, est. speed input: 105.36 toks/s, output: 28.72 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:14<00:00, 14.45s/it, est. speed input: 201.19 toks/s, output: 27.14 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.91s/it, est. speed input: 70.03 toks/s, output: 29.70 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:27<00:00, 27.16s/it, est. speed input: 28.28 toks/s, output: 30.45 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:21<00:00, 21.71s/it, est. speed input: 34.65 toks/s, output: 30.32 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:26<00:00, 26.38s/it, est. speed input: 29.08 toks/s, output: 30.44 toks/s]



Processed prompts: 100%|██████████████| 1/1 [00:14<00:00, 14.60s/it, est. speed input: 102.23 toks/s, output: 29.05 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.40s/it, est. speed input: 42.29 toks/s, output: 30.08 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:21<00:00, 21.86s/it, est. speed input: 29.96 toks/s, output: 30.38 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.10s/it, est. speed input: 33.46 toks/s, output: 30.37 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:25<00:00, 25.04s/it, est. speed input: 26.13 toks/s, output: 30.44 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:05<00:00,  5.66s/it, est. speed input: 69.42 toks/s, output: 29.32 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:09<00:00,  9.04s/it, est. speed input: 43.92 toks/s, output: 29.98 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.74s/it, est. speed input: 27.73 toks/s, output: 30.42 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.26s/it, est. speed input: 22.21 toks/s, output: 30.47 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:09<00:00,  9.03s/it, est. speed input: 29.35 toks/s, output: 30.23 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.89s/it, est. speed input: 47.09 toks/s, output: 30.09 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.38s/it, est. speed input: 56.97 toks/s, output: 29.90 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:21<00:00, 21.59s/it, est. speed input: 31.92 toks/s, output: 30.39 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.79s/it, est. speed input: 41.95 toks/s, output: 30.15 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:08<00:00,  8.18s/it, est. speed input: 113.17 toks/s, output: 28.84 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.25s/it, est. speed input: 82.69 toks/s, output: 29.43 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.15s/it, est. speed input: 64.26 toks/s, output: 29.76 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.43s/it, est. speed input: 61.91 toks/s, output: 29.88 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.50s/it, est. speed input: 78.96 toks/s, output: 29.48 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:09<00:00,  9.14s/it, est. speed input: 136.53 toks/s, output: 28.58 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.77s/it, est. speed input: 90.85 toks/s, output: 29.34 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:24<00:00, 24.29s/it, est. speed input: 50.84 toks/s, output: 30.01 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:17<00:00, 17.75s/it, est. speed input: 70.45 toks/s, output: 29.70 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:08<00:00,  8.69s/it, est. speed input: 70.58 toks/s, output: 29.59 toks/s]



...

Processed prompts: 100%|███████████████| 1/1 [00:08<00:00,  8.91s/it, est. speed input: 46.82 toks/s, output: 30.09 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.31s/it, est. speed input: 35.48 toks/s, output: 30.35 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:08<00:00,  8.04s/it, est. speed input: 51.77 toks/s, output: 29.99 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:09<00:00,  9.04s/it, est. speed input: 90.38 toks/s, output: 29.43 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:22<00:00, 22.16s/it, est. speed input: 37.05 toks/s, output: 30.46 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:17<00:00, 17.81s/it, est. speed input: 45.22 toks/s, output: 30.33 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.59s/it, est. speed input: 56.20 toks/s, output: 30.09 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:05<00:00,  5.22s/it, est. speed input: 59.00 toks/s, output: 29.69 toks/s]



Processed prompts: 100%|██████████████| 1/1 [00:06<00:00,  6.06s/it, est. speed input: 110.06 toks/s, output: 28.87 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.18s/it, est. speed input: 41.47 toks/s, output: 30.34 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:21<00:00, 21.59s/it, est. speed input: 30.34 toks/s, output: 30.57 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.53s/it, est. speed input: 53.47 toks/s, output: 30.09 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.01s/it, est. speed input: 65.49 toks/s, output: 29.98 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.68s/it, est. speed input: 62.58 toks/s, output: 30.05 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:20<00:00, 20.57s/it, est. speed input: 40.84 toks/s, output: 30.43 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.37s/it, est. speed input: 75.19 toks/s, output: 29.81 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:04<00:00,  4.83s/it, est. speed input: 56.14 toks/s, output: 29.62 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:06<00:00,  6.90s/it, est. speed input: 39.86 toks/s, output: 30.15 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.68s/it, est. speed input: 22.17 toks/s, output: 30.65 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:07<00:00,  7.35s/it, est. speed input: 37.32 toks/s, output: 30.23 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:08<00:00,  8.84s/it, est. speed input: 36.90 toks/s, output: 30.33 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.07s/it, est. speed input: 27.35 toks/s, output: 30.58 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.87s/it, est. speed input: 22.64 toks/s, output: 30.71 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.35s/it, est. speed input: 26.65 toks/s, output: 30.61 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.99s/it, est. speed input: 73.06 toks/s, output: 29.86 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:10<00:00, 10.44s/it, est. speed input: 117.76 toks/s, output: 29.03 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.97s/it, est. speed input: 82.39 toks/s, output: 29.67 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.61s/it, est. speed input: 77.96 toks/s, output: 29.72 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.98s/it, est. speed input: 88.15 toks/s, output: 29.55 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:09<00:00,  9.43s/it, est. speed input: 124.60 toks/s, output: 28.87 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.54s/it, est. speed input: 81.03 toks/s, output: 29.65 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:17<00:00, 17.12s/it, est. speed input: 67.89 toks/s, output: 29.85 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.72s/it, est. speed input: 26.10 toks/s, output: 30.59 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.55s/it, est. speed input: 32.90 toks/s, output: 30.43 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.34s/it, est. speed input: 54.00 toks/s, output: 30.08 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:18<00:00, 18.31s/it, est. speed input: 36.60 toks/s, output: 30.43 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:20<00:00, 20.41s/it, est. speed input: 32.04 toks/s, output: 30.52 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:17<00:00, 17.66s/it, est. speed input: 37.89 toks/s, output: 30.42 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.83s/it, est. speed input: 91.91 toks/s, output: 29.47 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.90s/it, est. speed input: 67.07 toks/s, output: 29.94 toks/s]

